In [2]:
#if you still need to install beautifulsoup4

! pip install requests beautifulsoup4 lxml

In [7]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# Basis-URL
base_url = "https://www.ots.at/suche"

# Angepasste Anfrage-Parameter
params = {
    "query": "",
    "seite": 1,
    "emittentId": 4468,  # Angepasste Emittent-ID
    "startDate": 1482015601,  # Angepasster Startzeitpunkt (Unix-Zeitstempel)
    "endDate": 1545173999,    # Angepasster Endzeitpunkt (Unix-Zeitstempel)
    "channel": "index",
    "attachment": ""
}

# Funktion zum Abrufen von Artikeldetails
def scrape_article_content(article_url):
    try:
        response = requests.get(article_url)
        if response.status_code != 200:
            print(f"Fehler beim Abrufen der Artikel-URL: {article_url}")
            return None, None, None

        soup = BeautifulSoup(response.text, "html.parser")
        # Titel
        title = soup.find("h1").get_text(strip=True)
        # Inhalt
        content_div = soup.find("div", {"itemprop": "articleBody"})
        content = content_div.get_text(strip=True) if content_div else "Kein Inhalt verfügbar"
        # Veröffentlichungsdatum
        date_meta = soup.find("meta", itemprop="datePublished")
        date_time = soup.find("time", {"itemprop": "datePublished"})
        date = (
            date_meta["content"] if date_meta and date_meta.has_attr("content") 
            else date_time["datetime"] if date_time and date_time.has_attr("datetime") 
            else date_time.get_text(strip=True) if date_time 
            else "Kein Datum verfügbar"
        )
        return title, content, date
    except Exception as e:
        print(f"Fehler beim Abrufen der Details: {e}")
        return None, None, None

# Funktion zum Scrapen einer Seite
def scrape_page(page_number):
    params["seite"] = page_number
    response = requests.get(base_url, params=params)
    if response.status_code != 200:
        print(f"Fehler beim Abrufen der Seite {page_number}")
        return []

    soup = BeautifulSoup(response.text, "html.parser")
    articles = soup.find_all("mat-card")
    data = []

    for article in articles:
        try:
            # Titel
            title_tag = article.find("h1", class_="display-3")
            title = title_tag.get_text(strip=True) if title_tag else "Kein Titel"

            # Teaser
            teaser_tag = article.find("p", class_="lead")
            teaser = teaser_tag.get_text(strip=True) if teaser_tag else "Kein Teaser"

            # Link
            link_tag = article.find("a", class_="link-detailed-view")
            link = f"https://www.ots.at{link_tag['href']}" if link_tag else "Kein Link"

            # Details scrapen
            article_title, article_content, article_date = scrape_article_content(link)

            # Daten hinzufügen
            data.append({
                "Titel": title,
                "Teaser": teaser,
                "Datum": article_date,
                "Link": link,
                "Inhalt": article_content
            })

        except Exception as e:
            print(f"Fehler beim Verarbeiten eines Artikels: {e}")
            continue

    return data

# Hauptfunktion für mehrere Seiten
def scrape_all_pages(start_page=1, end_page=5):
    all_data = []
    for page in range(start_page, end_page + 1):
        print(f"Scraping Seite {page}...")
        page_data = scrape_page(page)
        all_data.extend(page_data)
        time.sleep(2)  # Wartezeit zwischen Anfragen

    return all_data

# Ergebnisse speichern
def save_to_csv(data, filename="ots_scraper_data_emittent_4468.csv"):
    df = pd.DataFrame(data)
    df.to_csv(filename, index=False, encoding="utf-8")
    print(f"Daten erfolgreich gespeichert in {filename}")

# # Ausführung
# if __name__ == "__main__":
#     scraped_data = scrape_all_pages(start_page=1, end_page=5)  # Beispiel: erste 5 Seiten
#     if scraped_data:
#         save_to_csv(scraped_data)
#     else:
#         print("Keine Daten gescrapt.")


In [6]:
df_fpoe = pd.read_csv("ots_scraper_data_emittent_4468.csv")

display(df_fpoe)

,Titel,Teaser,Datum,Link,Inhalt
0,FPÖ-Wurm: „Modernisierung des VKI wurde heute ...,„Stärkung der Konsumentenrechte steht im Vorde...,2018-12-18T13:14:32+01:00,https://www.ots.at/presseaussendung/OTS_201812...,„Stärkung der Konsumentenrechte steht im Vorde...
1,Rapid-Fans: AUF/FPÖ-Herbert: „Polizei hat rech...,„Unsere Exekutivbeamten üben einen harten und ...,2018-12-18T10:43:33+01:00,https://www.ots.at/presseaussendung/OTS_201812...,„Unsere Exekutivbeamten üben einen harten und ...
2,FPÖ-Gudenus: „In der SPÖ gärt es gewaltig“,„Wortmeldung Doskozils zur Mindestsicherung is...,2018-12-18T10:25:14+01:00,https://www.ots.at/presseaussendung/OTS_201812...,„Wortmeldung Doskozils zur Mindestsicherung is...
3,Dinghofer-Institut verleiht Medienpreis an „Zu...,Widerstand gegen „political correctness” ein w...,2018-12-18T09:39:12+01:00,https://www.ots.at/presseaussendung/OTS_201812...,Widerstand gegen „political correctness” ein w...
4,"FPÖ-Schrangl an Becher: ""Rote Weihnachtsgesche...","""SPÖ übt sich in doppelbödigen Taschenspielert...",2018-12-17T13:50:26+01:00,https://www.ots.at/presseaussendung/OTS_201812...,"""SPÖ übt sich in doppelbödigen Taschenspielert..."
5,FPÖ-Hafenecker: „Künstlich inszenierte Großdem...,Wien (OTS) - „Die Demonstration am vergangenen...,2018-12-17T11:37:16+01:00,https://www.ots.at/presseaussendung/OTS_201812...,Wien (OTS) -„Die Demonstration am vergangenen ...
6,FPÖ-Hafenecker zu Hacker: „Vergleiche Mindests...,"Wien (OTS) - „Es ist wirklich schockierend, mi...",2018-12-16T11:23:19+01:00,https://www.ots.at/presseaussendung/OTS_201812...,"Wien (OTS) -„Es ist wirklich schockierend, mit..."
7,Vilimsky: „Linke Demo war rot-grüne Sabotage d...,Wien (OTS) - „Die Geschäfte machen den größten...,2018-12-16T10:08:34+01:00,https://www.ots.at/presseaussendung/OTS_201812...,Wien (OTS) -„Die Geschäfte machen den größten ...
8,Vilimsky: Nach Hacker jetzt Schieder mit skand...,SPÖ nur mehr außer Rand und Band,2018-12-15T17:02:03+01:00,https://www.ots.at/presseaussendung/OTS_201812...,SPÖ nur mehr außer Rand und BandWien (OTS) -Na...
9,FPÖ-Lasar schockiert über NS-Vergleich von SPÖ...,Hacker legt nach und spricht von „Ariernachwei...,2018-12-15T10:33:33+01:00,https://www.ots.at/presseaussendung/OTS_201812...,Hacker legt nach und spricht von „Ariernachwei...


In [8]:
# Ausführung: Scrape data in steps
for i in range(2):  # Adjust the range as needed to cover all steps
    x = i * 100  # Calculate the starting page for this iteration
    if __name__ == "__main__":
        print(f"Scraping pages {x + 1} to {x + 100}...")
        scraped_data = scrape_all_pages(start_page=x + 1, end_page=x + 100)  
        if scraped_data:
            # Save to a unique file for each iteration
            filename = f"ots_scraper_data_fpoe_part_{i + 1}.csv"
            save_to_csv(scraped_data, filename)
        else:
            print(f"Keine Daten gescrapt für Seiten {x + 1} bis {x + 100}.")


Scraping pages 1 to 100...
Scraping Seite 1...
Scraping Seite 2...
Scraping Seite 3...
Scraping Seite 4...
Scraping Seite 5...
Scraping Seite 6...
Scraping Seite 7...
Scraping Seite 8...
Scraping Seite 9...
Scraping Seite 10...
Scraping Seite 11...
Scraping Seite 12...
Scraping Seite 13...
Scraping Seite 14...
Scraping Seite 15...
Scraping Seite 16...
Scraping Seite 17...
Scraping Seite 18...
Scraping Seite 19...
Scraping Seite 20...
Scraping Seite 21...
Scraping Seite 22...
Scraping Seite 23...
Scraping Seite 24...
Scraping Seite 25...
Scraping Seite 26...
Scraping Seite 27...
Scraping Seite 28...
Scraping Seite 29...
Scraping Seite 30...
Scraping Seite 31...
Scraping Seite 32...
Scraping Seite 33...
Scraping Seite 34...
Scraping Seite 35...
Scraping Seite 36...
Scraping Seite 37...
Scraping Seite 38...
Scraping Seite 39...
Scraping Seite 40...
Scraping Seite 41...
Scraping Seite 42...
Scraping Seite 43...
Scraping Seite 44...
Scraping Seite 45...
Scraping Seite 46...
Scraping Seite 4

In [11]:
df_fpoe = pd.read_csv("ots_scraper_data_fpoe_part_1.csv")

display(df_fpoe)



,Titel,Teaser,Datum,Link,Inhalt
0,FPÖ-Wurm: „Modernisierung des VKI wurde heute ...,„Stärkung der Konsumentenrechte steht im Vorde...,2018-12-18T13:14:32+01:00,https://www.ots.at/presseaussendung/OTS_201812...,„Stärkung der Konsumentenrechte steht im Vorde...
1,Rapid-Fans: AUF/FPÖ-Herbert: „Polizei hat rech...,„Unsere Exekutivbeamten üben einen harten und ...,2018-12-18T10:43:33+01:00,https://www.ots.at/presseaussendung/OTS_201812...,„Unsere Exekutivbeamten üben einen harten und ...
2,FPÖ-Gudenus: „In der SPÖ gärt es gewaltig“,„Wortmeldung Doskozils zur Mindestsicherung is...,2018-12-18T10:25:14+01:00,https://www.ots.at/presseaussendung/OTS_201812...,„Wortmeldung Doskozils zur Mindestsicherung is...
3,Dinghofer-Institut verleiht Medienpreis an „Zu...,Widerstand gegen „political correctness” ein w...,2018-12-18T09:39:12+01:00,https://www.ots.at/presseaussendung/OTS_201812...,Widerstand gegen „political correctness” ein w...
4,"FPÖ-Schrangl an Becher: ""Rote Weihnachtsgesche...","""SPÖ übt sich in doppelbödigen Taschenspielert...",2018-12-17T13:50:26+01:00,https://www.ots.at/presseaussendung/OTS_201812...,"""SPÖ übt sich in doppelbödigen Taschenspielert..."
...,...,...,...,...,...
995,ÖVP-Kurz-Geheimpapiere: FPÖ-Kickl: Es ist Zeit...,FPÖ fordert nochmals eidesstattliche Erklärung...,2017-09-20T13:33:22+02:00,https://www.ots.at/presseaussendung/OTS_201709...,FPÖ fordert nochmals eidesstattliche Erklärung...
996,FPÖ-Mölzer: Parteiwerbung am Bundesgymnasium f...,„Grüne werben im 60. Jahresbericht des BG/BRG ...,2017-09-20T09:57:49+02:00,https://www.ots.at/presseaussendung/OTS_201709...,„Grüne werben im 60. Jahresbericht des BG/BRG ...
997,Vilimsky: „Wir brauchen strengere Lobbying-Reg...,„Legislativer Fußabdruck“: ÖVP stimmte gegen S...,2017-09-20T09:35:31+02:00,https://www.ots.at/presseaussendung/OTS_201709...,„Legislativer Fußabdruck“: ÖVP stimmte gegen S...
998,FPÖ-Rauch zu Glyphosat-Neuzulassung: ÖVP muss ...,FPÖ spricht sich gegen weitere Zulassung von G...,2017-09-19T15:33:22+02:00,https://www.ots.at/presseaussendung/OTS_201709...,FPÖ spricht sich gegen weitere Zulassung von G...


In [12]:
import pandas as pd
import os

# Name der Ausgabe-Datei
output_filename = "ots_scraper_data_fpoe_combined.csv"

def combine_csv_files(output_filename):
    # Aktuelles Verzeichnis des Notebooks
    current_directory = os.getcwd()

    # Liste, um alle DataFrames zu speichern
    all_data = []

    # Iteriere durch alle Dateien im aktuellen Verzeichnis
    for filename in os.listdir(current_directory):
        if filename.startswith("ots_scraper_data_fpoe_part_") and filename.endswith(".csv"):
            file_path = os.path.join(current_directory, filename)
            print(f"Verarbeite Datei: {file_path}")
            # Lade die CSV-Datei und füge sie zur Liste hinzu
            df = pd.read_csv(file_path)
            all_data.append(df)

    # Kombiniere alle DataFrames in ein einziges DataFrame
    if all_data:
        combined_df = pd.concat(all_data, ignore_index=True)
        # Speichere das kombinierte DataFrame in einer neuen CSV-Datei
        combined_df.to_csv(output_filename, index=False, encoding="utf-8")
        print(f"Alle Dateien erfolgreich kombiniert in: {output_filename}")
    else:
        print("Keine Dateien gefunden, die kombiniert werden können.")

# Funktion ausführen
combine_csv_files(output_filename)


Verarbeite Datei: c:\Users\walchhth\OneDrive - Styria-IT Solutions GmbH & Co KG\Dokumente\KLZ - TOM\CSS - Master\1. Semester\Foundations of CSS\Group_Project\data\data_fpoe\ots_scraper_data_fpoe_part_1.csv
Verarbeite Datei: c:\Users\walchhth\OneDrive - Styria-IT Solutions GmbH & Co KG\Dokumente\KLZ - TOM\CSS - Master\1. Semester\Foundations of CSS\Group_Project\data\data_fpoe\ots_scraper_data_fpoe_part_2.csv
Alle Dateien erfolgreich kombiniert in: ots_scraper_data_fpoe_combined.csv
